In [1]:
from fyers_apiv3 import fyersModel
from datetime import datetime , timedelta
import pandas as pd
import pytz

# Genrate Token

In [2]:
client_id = "MACO3YJA7I-100"  # Replace with your client ID
secret_key = "N7SXUFQG91"  # Replace with your secret key
redirect_uri = "https://trade.fyers.in/api-login/redirect-uri/index.html"  # Replace with your redirect URI
response_type = "code" 
grant_type = "authorization_code"  
state = "sample_state"

### Genearte Authcode

In [6]:
# # # Generate Auth Code
# session = fyersModel.SessionModel(
#     client_id=client_id,
#     secret_key=secret_key,
#     redirect_uri=redirect_uri,
#     response_type=response_type
# )
# genrate_authcode = session.generate_authcode()
# print(genrate_authcode)

https://api-t1.fyers.in/api/v3/generate-authcode?client_id=MACO3YJA7I-100&redirect_uri=https%3A%2F%2Ftrade.fyers.in%2Fapi-login%2Fredirect-uri%2Findex.html&response_type=code&state=None


In [7]:
auth_code = "eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJpYXQiOjE3MDI5NjkzNDEsImV4cCI6MTcwMjk5OTM0MSwibmJmIjoxNzAyOTY4NzQxLCJhdWQiOiJbXCJ4OjBcIiwgXCJ4OjFcIiwgXCJ4OjJcIiwgXCJkOjFcIiwgXCJkOjJcIiwgXCJ4OjFcIiwgXCJ4OjBcIl0iLCJzdWIiOiJhdXRoX2NvZGUiLCJkaXNwbGF5X25hbWUiOiJZSjAwMTI5Iiwib21zIjoiSzEiLCJoc21fa2V5IjoiMjdlMDQ2OGE4MGJkYmVmMGQyYTg3M2MxMmNlN2YyZGEyMjA1NzQ0ZTIwYTg1MDI0YjhlMDA4NDciLCJub25jZSI6IiIsImFwcF9pZCI6Ik1BQ08zWUpBN0kiLCJ1dWlkIjoiMzEwNjE3ZTRiNmE1NDg1N2FiNmFiMGQ0NGM5NzljNmMiLCJpcEFkZHIiOiIwLjAuMC4wIiwic2NvcGUiOiIifQ.Lw54JoBH28CihxWWWBcuDVieHo5sDVY_6_yI4N1DIVQ"

### Set Session & Get TOKEN

In [8]:
session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key, 
    redirect_uri=redirect_uri, 
    response_type=response_type, 
    grant_type=grant_type
)
session.set_token(auth_code)
response = session.generate_token()
access_token = response['access_token']

# Login 

In [9]:
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

In [10]:
fyers.get_profile()

{'s': 'ok',
 'code': 200,
 'message': '',
 'data': {'fy_id': 'YJ00129',
  'name': 'JAYKUMAR DHANSUKHBHAI PATEL',
  'image': None,
  'display_name': None,
  'pin_change_date': '18-12-2023 16:00:58',
  'email_id': 'justjayy19@gmail.com',
  'pwd_change_date': None,
  'PAN': '---------',
  'mobile_number': '7069668308',
  'totp': True,
  'pwd_to_expire': 90}}

In [11]:
# dir(fyers)

# Get Historical Data

In [12]:
def n_day_date(n):
    current_date = datetime.now()
    n_days_ago = current_date - timedelta(days=n)
    return n_days_ago.strftime('%Y-%m-%d')

In [13]:
def Historical_Data(Symbol,TimeFrame,Start_Date,End_Date):
    data = {
                "symbol":Symbol,
                "resolution":TimeFrame,
                "date_format":"1",
                "range_from":Start_Date,
                "range_to":End_Date,
                "cont_flag":"0"
            }
    row_data =  fyers.history(data=data)
    df = pd.DataFrame.from_dict(row_data['candles'])
    columns_name = ['Datetime','Open','High','Low','Close','Volume']
    df.columns = columns_name
    df['Datetime'] = pd.to_datetime(df['Datetime'], unit='s')
    df['Datetime'] = df['Datetime'].dt.tz_localize(pytz.utc).dt.tz_convert('Asia/Kolkata')
    df['Datetime'] = df['Datetime'].dt.tz_localize(None)

    return df

Symbol = "NSE:SBIN-EQ"
TimeFrame = "5"

Start_Date = n_day_date(5)
End_Date = datetime.now().strftime('%Y-%m-%d')
data = Historical_Data(Symbol,TimeFrame,Start_Date,End_Date)

In [14]:
data

,Datetime,Open,High,Low,Close,Volume
0,2023-12-14 09:15:00,626.00,627.60,625.00,625.20,1294993
1,2023-12-14 09:20:00,625.20,625.50,623.50,624.50,408915
2,2023-12-14 09:25:00,624.40,625.95,624.25,625.00,291613
3,2023-12-14 09:30:00,624.95,625.50,623.30,623.50,427222
4,2023-12-14 09:35:00,623.50,624.30,623.10,624.10,339303
...,...,...,...,...,...,...
260,2023-12-19 12:10:00,652.20,652.85,651.70,651.95,124062
261,2023-12-19 12:15:00,651.90,654.60,651.75,653.90,343526
262,2023-12-19 12:20:00,653.80,659.50,653.80,659.45,1165251
263,2023-12-19 12:25:00,659.45,659.60,657.50,657.85,425063


# GET STOCK DATA

In [15]:
import pandas as pd
df = pd.read_csv("ind_nifty500list.csv")
def Get_Symbol(row):
    Symbol= row['Symbol'] 
    Series = "EQ"
    Ex = "NSE"
    Ticker = f"{Ex}:{Symbol}-{Series}"
    return Ticker
df['Tickers'] = df.apply(Get_Symbol,axis=1)
# df = df.sample(100)

In [16]:
def assess_liquidity_percentage(stock_data):
    try:
        data = {"symbols":stock_data['Tickers']}
        response = fyers.quotes(data=data)
        stock_data = response['d'][0]['v']
        
        volume_threshold = 1000000  # Adjust based on your criteria
        spread_threshold = 0.1  # Adjust based on your criteria
        price_difference_threshold = 5  # Adjust based on your criteria
    
        # 1. Volume
        volume_percentage = min(stock_data['volume'] / volume_threshold * 100, 100)
    
        # 2. Bid-Ask Spread
        spread_percentage = max(1 - stock_data['spread'] / spread_threshold, 0) * 100
    
        # 3. Price Alignment
        price_difference = abs(stock_data['open_price'] - stock_data['lp'])
        price_alignment_percentage = max(1 - price_difference / price_difference_threshold, 0) * 100
    
        # Calculate the overall percentage
        overall_percentage = (volume_percentage + spread_percentage + price_alignment_percentage) / 3
        return overall_percentage,stock_data['lp'], stock_data['chp']
    except:
        return None,None,None



In [17]:
df[['Status','Price','ChnagePr']] = df.apply(assess_liquidity_percentage,axis=1,result_type='expand')

In [18]:
filter_df1 = df[ (df['Status'] >= 50) & (df['Price'] <= 100 ) & (df['ChnagePr'] >= 0.7 )  ].sort_values(by='ChnagePr', ascending=False)

In [21]:
filter_df1

,Company Name,Industry,Symbol,Series,ISIN Code,Tickers,Status,Price,ChnagePr
232,Indian Overseas Bank,Financial Services,IOB,EQ,INE565A01014,NSE:IOB-EQ,74.333333,46.05,3.02
98,Central Bank of India,Financial Services,CENTRALBK,EQ,INE483A01010,NSE:CENTRALBK-EQ,73.666667,52.05,2.97
436,TV18 Broadcast Ltd.,Media Entertainment & Publication,TV18BRDCST,EQ,INE886H01027,NSE:TV18BRDCST-EQ,77.000000,50.95,2.31
28,Alok Industries Ltd.,Textiles,ALOKINDS,EQ,INE270A01029,NSE:ALOKINDS-EQ,81.333333,23.00,1.77
486,Vodafone Idea Ltd.,Telecommunication,IDEA,EQ,INE669E01016,NSE:IDEA-EQ,82.000000,14.35,1.77
198,HFCL Ltd.,Telecommunication,HFCL,EQ,INE548A01028,NSE:HFCL-EQ,77.333333,79.45,1.74
218,IDFC First Bank Ltd.,Financial Services,IDFCFIRSTB,EQ,INE092T01019,NSE:IDFCFIRSTB-EQ,83.000000,90.65,1.00
333,Network18 Media & Investments Ltd.,Media Entertainment & Publication,NETWORK18,EQ,INE870H01013,NSE:NETWORK18-EQ,63.000000,86.80,0.70


In [30]:
import yfinance as yf
import pandas as pd
import ta
import pandas_ta as pdta

def Scan_stock(row):
    try:
        ticker = yf.Ticker(f"{row['Symbol']}.BO")
        df = ticker.history(period="500d", interval="1d")
        supertrend = pdta.supertrend(df['High'], df['Low'], df['Close'], length=7, multiplier=3)
        df['Trend'] = supertrend['SUPERTd_7_3.0']
        df['EMA_5'] = ta.trend.ema_indicator(df['Close'], window=5)
        df['EMA_200'] = ta.trend.ema_indicator(df['Close'], window=200)
        df['EMA_200'] = ta.trend.ema_indicator(df['Close'], window=200)
        df['RSI'] = pdta.momentum.rsi(df['Close'], window=14)
        df['Trend_signal'] = ((df['Trend'] == 1) & (df['Trend'].shift(1) == -1))
        df['RSI_signal'] = df['RSI'] <= 50
        df['Volume_signal'] = df['Volume'] > df['Volume'].rolling(window=20).mean()
        df['Buy'] = (df[['Trend_signal', 'RSI_signal', 'Volume_signal']].sum(axis=1) >= 2)
        df['SL'] = df['Close'].shift(1).rolling(window=60).min() 
        df['Target'] =  df['Close']  + (df['Close'] - df['SL'])*1.5
        return df.iloc[-1]['Buy'] ,df.iloc[-1]['SL'],df.iloc[-1]['Target']
    except Exception as e:
        return None, None, None

In [31]:
filter_df1[['Buy_Status','SL','Target']] = filter_df1.apply(Scan_stock,axis=1,result_type='expand')

ALOKINDS.BO: No data found, symbol may be delisted


In [ ]:
# filter_df1[filter_df1['Buy_Status']].to_csv("today_tred.csv")

In [32]:
filter_df1

,Company Name,Industry,Symbol,Series,ISIN Code,Tickers,Status,Price,ChnagePr,Buy_Status,SL,Target
232,Indian Overseas Bank,Financial Services,IOB,EQ,INE565A01014,NSE:IOB-EQ,74.333333,46.05,3.02,False,37.209999,59.685003
98,Central Bank of India,Financial Services,CENTRALBK,EQ,INE483A01010,NSE:CENTRALBK-EQ,73.666667,52.05,2.97,False,42.549999,66.325005
436,TV18 Broadcast Ltd.,Media Entertainment & Publication,TV18BRDCST,EQ,INE886H01027,NSE:TV18BRDCST-EQ,77.000000,50.95,2.31,False,40.450001,65.874996
28,Alok Industries Ltd.,Textiles,ALOKINDS,EQ,INE270A01029,NSE:ALOKINDS-EQ,81.333333,23.00,1.77,None,NaN,NaN
486,Vodafone Idea Ltd.,Telecommunication,IDEA,EQ,INE669E01016,NSE:IDEA-EQ,82.000000,14.35,1.77,False,10.730000,19.705000
198,HFCL Ltd.,Telecommunication,HFCL,EQ,INE548A01028,NSE:HFCL-EQ,77.333333,79.45,1.74,False,63.389999,103.714993
218,IDFC First Bank Ltd.,Financial Services,IDFCFIRSTB,EQ,INE092T01019,NSE:IDFCFIRSTB-EQ,83.000000,90.65,1.00,False,81.110001,105.060005
333,Network18 Media & Investments Ltd.,Media Entertainment & Publication,NETWORK18,EQ,INE870H01013,NSE:NETWORK18-EQ,63.000000,86.80,0.70,False,63.209999,121.785000
